In [1]:
import os
from aaai_experiments import *

In [2]:
PATH = 'C:\\Users\\User\\Jupyter Projects\\SKTR'
os.chdir(PATH)

## A*, A* incremental, and REACH evaluations

In [14]:
# Parameters for `load_and_preprocess_log`
df_name = 'bpi_2012.csv'
min_len = 80
max_len = None
n_traces = None

In [15]:
df, map_dict = load_and_preprocess_log(df_name, min_len=80, max_len=None, n_traces=None, random_seed=304)

In [16]:
random_seed = 304

df_sample = sample_unique_traces(df, exclude_list=cases_for_discovery, n_traces=15, random_seed=random_seed)

In [17]:
# Parameters for `compare_search_algorithms`
allow_intersection = True
read_model_from_file = False
activity_mapping_dict = map_dict
only_return_model = False
algorithms=['astar', 'reach']

In [21]:
compare_search_algorithms(
    df=df_sample,
    df_name=df_name,
    train_cases=cases_for_discovery,
    n_test_traces=None,
    allow_intersection=allow_intersection,
    read_model_from_file=read_model_from_file,
    activity_mapping_dict=activity_mapping_dict,
    only_return_model=only_return_model,
    algorithms=algorithms,
)

The train cases are: ['5087', '5440', '11482', '19165', '22701'] ... [and 10 more]

The test cases are: ['23', '31266', '37186', '42549', '43424'] ... [and 10 more]



KeyError: "None of [Index(['5087', '5440', '11482', '19165', '22701', '23778', '26537', '66048',\n       '77689', '98546', '112277', '116264', '141752', '230433', '242371'],\n      dtype='object', name='case:concept:name')] are in the [index]"

## Running Horizon (+ heuristics) evaluations

In [3]:
df_name = 'bpi_2012.csv'
model_path = ''
n_train_traces = 10
n_test_traces = None
window_lengths_lst = [15, 30, 50]
n_final_markings_lst = [1]
min_len = 80
max_len = None
use_heuristics = False 
read_model_from_file = False
allow_intersection = True

In [5]:
res = compare_window_based_baselines(
    df_name=df_name,
    n_train_traces=n_train_traces,
    n_test_traces=n_test_traces,
    window_lengths_lst=window_lengths_lst,
    n_final_markings_lst=n_final_markings_lst,
    min_len=min_len,
    max_len=max_len,
    use_heuristics=use_heuristics,
    read_model_from_file=read_model_from_file,
    model_path=model_path,
    allow_intersection=allow_intersection
)

## Video Datasets evaluations

### Our approach

In [1]:
n_train_traces = 8
n_test_traces = None
window_lengths_lst = [20]
n_final_markings_lst = [1]
use_heuristics = True 
allow_intersection = False
random_seed=101

In [4]:
# Define the path and file names
df_path = r'C:\Users\User\Jupyter Projects\Research_2\Datasets\Long_Traces'
df_names = ['gtea_target.pkl', '50salads_target.pkl', 'breakfast_target.pkl']

# Parameters for different datasets
dataset_params = {
    'gtea_target.pkl': 8,
    '50salads_target.pkl': 8,
    'breakfast_target.pkl': 15    
}

# Loop over each dataset
for df_name in df_names:
    print(f"Processing dataset: {df_name}")
    
    # Process the pickle file
    df = process_pickle_file(os.path.join(df_path, df_name))
    
    # Adjust the number of training traces based on the dataset
    n_train_traces = dataset_params[df_name]
    
    # Call the compare function with the processed DataFrame
    res = compare_window_based_baselines(
        df_name=df,
        n_train_traces=n_train_traces,
        n_test_traces=n_test_traces,
        window_lengths_lst=window_lengths_lst,
        n_final_markings_lst=n_final_markings_lst,
        use_heuristics=use_heuristics,
        allow_intersection=allow_intersection,
        random_seed=random_seed
    )
    
    # Define the path to save the results
    result_save_path = os.path.join(df_path, f"{df_name.split('_')[0]}_results.csv")
    
    # Save the results after processing the dataset
    save_results(res, result_save_path)
    print(f"Results saved to {result_save_path}")

### Baselines

In [2]:
# Parameters for `compare_search_algorithms`
n_test_traces = None
allow_intersection = True
read_model_from_file = False
only_return_model = False
algorithms=['astar', 'astar_extended', 'reach']
return_results=True

In [1]:
# Define the path and file names
df_path = r'C:\Users\User\Jupyter Projects\Research_2\Datasets\Long_Traces'
df_names = ['gtea_target.pkl', '50salads_target.pkl', 'breakfast_target.pkl']
save_path = r'C:\path\to\save\directory'  # Update with your actual save path

# Parameters for different datasets
dataset_params = {
    'gtea_target.pkl': 8,
    '50salads_target.pkl': 8,
    'breakfast_target.pkl': 15    
}

# Loop over each dataset
for df_name in df_names:
    print(f"Processing dataset: {df_name}")
    
    # Set time limit based on dataset name
    if df_name == '50salads_target.pkl':
        time_limit = 120
    else:  # for 'gtea_target.pkl' and 'breakfast_target.pkl'
        time_limit = 15
    
    # Process the pickle file
    df = process_pickle_file(os.path.join(df_path, df_name))
    
    # Adjust the number of training traces based on the dataset
    n_train_traces = dataset_params[df_name]
    
    # Call the compare function with the processed DataFrame
    results = compare_search_algorithms(
        df=df,
        df_name=df_name,
        n_train_traces=n_train_traces,
        n_test_traces=n_test_traces,
        allow_intersection=allow_intersection,
        read_model_from_file=read_model_from_file,
        only_return_model=only_return_model,
        algorithms=algorithms,
        time_limit=time_limit,  # Use the time limit set above
        return_results=return_results
    )
    
    # Save each algorithm's results to a separate pickle file
    for alg, alg_results in results.items():
        result_file = os.path.join(save_path, f"{df_name.split('_')[0]}_{alg}_results.csv")
        
        # Convert results to DataFrame, drop rows with all None values, and save
        results_df = pd.DataFrame(alg_results).dropna(how='any')
        results_df.to_csv(result_file, index=False)
        
        print(f"Results for algorithm '{alg}' saved to {result_file}")

## Results interpretation

In [17]:
baseline_path = 'C:\\Users\\User\\Jupyter Projects\\SKTR\\results\\baseline_algorithms\\prSomething\\A57'
os.chdir(baseline_path)

# Get the current working directory
directory = os.getcwd()

# Loop through each file in the current directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        # Read the CSV file into a DataFrame
        df = pd.read_csv(filename)

        if filename == 'pr-1-11-1244-A59_m55_l4_noise.csv_astar_results.csv':
            df_opt = df
        # Print the filename for context
        print(f"Summary statistics for {filename}:\n")
        
        # Display the description of the DataFrame
        print(df.describe())
        print("\n" + "-"*50 + "\n")  # Separator between files

Summary statistics for pr-8-11-1912-A57_m15_l1.csv_astar_results.csv:

           case_id         cost         time  nodes_popped
count  1985.000000  1985.000000  1985.000000   1985.000000
mean    999.651385     0.000013     0.194306     15.723929
std     577.309590     0.000005     0.075547      3.162016
min       0.000000     0.000010     0.130030     13.000000
25%     500.000000     0.000010     0.143390     14.000000
50%     999.000000     0.000010     0.144620     14.000000
75%    1500.000000     0.000020     0.265770     19.000000
max    1999.000000     0.000030     0.497610     27.000000

--------------------------------------------------

Summary statistics for pr-8-11-1912-A57_m15_l1.csv_reach_results.csv:

           case_id         cost         time  nodes_popped
count  1985.000000  1985.000000  1985.000000   1985.000000
mean    999.651385     0.000013     0.002315     31.447859
std     577.309590     0.000005     0.000845      6.324031
min       0.000000     0.000010     0.

C:\Users\User\miniconda3\envs\researchenv\Lib\site-packages\numpy\lib\function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Users\User\miniconda3\envs\researchenv\Lib\site-packages\numpy\lib\function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Users\User\miniconda3\envs\researchenv\Lib\site-packages\numpy\lib\function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


In [9]:
PATH = 'C:\\Users\\User\\Jupyter Projects\\SKTR\\new_results'
os.chdir(PATH)

# Get the current working directory
directory = os.getcwd()

# Loop through each file in the current directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        # Read the CSV file into a DataFrame
        df = pd.read_csv(filename)
        if filename == 'pr-1-11-1244-A59_m55_l4_noise_results_h.csv':
            df_h = df
        elif filename == 'pr-1-11-1244-A59_m55_l4_noise_results.csv':
            df_no_h = df
        else:
            continue

        print(f"Summary statistics for {filename}:\n")
        
        # Display the description of the DataFrame
        print(df.describe())
        print("\n" + "-"*50 + "\n")  # Separator between files

In [17]:
df_h.describe()

,case_id,window_15_markings_1_cost,window_15_markings_1_time,window_15_markings_1_nodes_opened,window_30_markings_1_cost,window_30_markings_1_time,window_30_markings_1_nodes_opened,window_50_markings_1_cost,window_50_markings_1_time,window_50_markings_1_nodes_opened
count,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000
mean,1030.987013,9.740343,0.095000,624.597403,9.298785,0.157245,1097.246753,8.000084,0.267440,1786.597403
std,573.438417,2.086313,0.050115,178.783481,2.045952,0.069215,376.405372,0.648886,0.082327,329.566592
min,4.000000,4.000080,0.033760,221.000000,4.000080,0.055930,404.000000,4.000080,0.094440,935.000000
25%,516.000000,8.000080,0.070180,495.000000,8.000080,0.096310,738.000000,8.000080,0.213210,1614.000000
50%,1097.000000,8.000100,0.094460,653.000000,8.000090,0.147430,1113.000000,8.000080,0.250370,1803.000000
75%,1468.000000,12.000080,0.109310,753.000000,12.000080,0.199690,1389.000000,8.000090,0.311660,2001.000000
max,1946.000000,14.000090,0.462410,1007.000000,14.000090,0.414490,1828.000000,12.000080,0.630400,2879.000000


In [23]:
len(df_no_h[df_no_h['window_15_markings_1_cost'] < 8.1]) / len(df_no_h)

0.5194805194805194

In [36]:
PATH = 'C:\\Users\\User\\Jupyter Projects\\SKTR'
os.chdir(PATH)

In [11]:
df = pd.read_csv('bpi_2012.csv_astar_results.csv')
df.describe()

,case_id,cost,time,nodes_popped
count,100.000000,100.000000,100.000000,100.000000
mean,108107.440000,0.400208,143.940960,10676.120000
std,76570.226697,2.344117,108.480666,7062.706342
min,73.000000,0.000090,27.362740,2377.000000
25%,30576.500000,0.000150,65.379945,5557.500000
50%,101750.000000,0.000200,106.904625,8155.000000
75%,172636.500000,0.000282,219.164965,13940.250000
max,246494.000000,22.000070,509.934630,28928.000000


In [12]:
df = pd.read_csv('bpi_2019.csv_astar_results.csv')
df.describe()

,case_id,cost,time,nodes_popped
count,100.000000,100.000000,100.000000,100.00000
mean,108107.440000,0.400208,144.550152,10668.37000
std,76570.226697,2.344117,109.169797,7045.59276
min,73.000000,0.000090,27.446910,2330.00000
25%,30576.500000,0.000150,65.327613,5553.00000
50%,101750.000000,0.000200,106.812605,8148.00000
75%,172636.500000,0.000282,218.805090,13952.50000
max,246494.000000,22.000070,501.202760,28899.00000


In [14]:
df = pd.read_csv('bpi_2012.csv_reach_results.csv')
df.describe()

,case_id,cost,time,nodes_popped
count,100.000000,100.000000,100.000000,100.00000
mean,108107.440000,0.400208,4.778222,13529.95000
std,76570.226697,2.344117,4.907632,12670.58875
min,73.000000,0.000090,0.604190,3417.00000
25%,30576.500000,0.000150,1.771275,6236.00000
50%,101750.000000,0.000200,2.848075,10162.50000
75%,172636.500000,0.000282,6.562143,15133.00000
max,246494.000000,22.000070,27.042750,89149.00000


In [15]:
df = pd.read_csv('bpi_2019.csv_reach_results.csv')
df.describe()

,case_id,cost,time,nodes_popped
count,100.000000,100.000000,100.000000,100.00000
mean,108107.440000,0.400208,4.761300,13535.81000
std,76570.226697,2.344117,4.905240,12708.55021
min,73.000000,0.000090,0.602570,3419.00000
25%,30576.500000,0.000150,1.724905,6238.25000
50%,101750.000000,0.000200,2.831160,10149.50000
75%,172636.500000,0.000282,6.514737,15119.50000
max,246494.000000,22.000070,27.197130,89550.00000


In [39]:
reach_gtea_res = pd.read_csv('breakfast_reach_results_new.csv')

In [40]:
reach_gtea_res.describe()

,cost,time,nodes_popped,case_id
count,198.000000,198.000000,198.000000,198.000000
mean,inf,16.634644,10682.055556,472.878788
std,NaN,20.076497,12378.673871,319.556872
min,0.000264,0.770555,2179.000000,0.000000
25%,0.000401,1.929781,3108.000000,125.250000
50%,0.000780,8.590468,5485.000000,504.500000
75%,1.000591,20.376747,11204.000000,733.750000
max,inf,104.921433,68796.000000,1007.000000


In [16]:
PATH = 'C:\\Users\\User\\Jupyter Projects\\SKTR\\results\\my_algorithm_results\\bpi'
os.chdir(PATH)

In [21]:
df_my_alg_2019_h = pd.read_csv('bpi_2019_results_h.csv')

In [22]:
df_my_alg_2012_h.describe()

,case_id,window_15_markings_1_cost,window_15_markings_1_time,window_15_markings_1_nodes_opened,window_30_markings_1_cost,window_30_markings_1_time,window_30_markings_1_nodes_opened,window_50_markings_1_cost,window_50_markings_1_time,window_50_markings_1_nodes_opened
count,881.000000,881.000000,881.000000,881.000000,881.000000,881.000000,881.000000,881.000000,881.000000,881.000000
mean,115388.500568,0.572291,1.246769,11943.558456,0.572291,1.465462,12248.953462,0.572291,1.716778,12649.673099
std,78619.541409,3.191903,1.110320,9098.069493,3.191903,1.306864,9088.479613,3.191903,1.555800,9028.253508
min,18.000000,0.000080,0.265890,3104.000000,0.000080,0.336800,3261.000000,0.000080,0.428200,3647.000000
25%,40577.000000,0.000160,0.593820,6658.000000,0.000160,0.702900,6999.000000,0.000160,0.842630,7695.000000
50%,116262.000000,0.000200,0.949090,9825.000000,0.000200,1.097110,10167.000000,0.000200,1.280490,10735.000000
75%,183620.000000,0.000280,1.616950,14630.000000,0.000280,1.875780,14819.000000,0.000280,1.961530,14998.000000
max,250991.000000,31.000450,12.218510,105169.000000,31.000450,13.925060,106382.000000,31.000450,16.851940,106697.000000


In [23]:
df_my_alg_2019 = pd.read_csv('bpi_2019_results.csv')

In [24]:
df_my_alg_2012.describe()

,case_id,window_15_markings_1_cost,window_15_markings_1_time,window_15_markings_1_nodes_opened,window_30_markings_1_cost,window_30_markings_1_time,window_30_markings_1_nodes_opened,window_50_markings_1_cost,window_50_markings_1_time,window_50_markings_1_nodes_opened
count,881.000000,881.000000,881.000000,881.000000,881.000000,881.000000,881.000000,881.000000,881.000000,881.000000
mean,115388.500568,0.572291,1.344906,18713.202043,0.572291,1.706535,18834.839955,0.572291,2.140471,19043.874007
std,78619.541409,3.191903,0.749147,9514.042693,3.191903,1.105122,9669.640676,3.191903,1.441975,9914.519415
min,18.000000,0.000080,0.444720,7364.000000,0.000080,0.543050,7364.000000,0.000080,0.663040,7364.000000
25%,40577.000000,0.000160,0.842770,13082.000000,0.000160,1.023780,13112.000000,0.000160,1.299140,13174.000000
50%,116262.000000,0.000200,1.187950,16745.000000,0.000200,1.329850,16798.000000,0.000200,1.703220,17038.000000
75%,183620.000000,0.000280,1.657310,22374.000000,0.000280,2.098840,22473.000000,0.000280,2.529580,22841.000000
max,250991.000000,31.000450,7.882720,108643.000000,31.000450,10.882440,111391.000000,31.000450,14.588240,112322.000000


In [21]:
def compute_dataset_stats(dataset_name):
    # Apply filtering if needed
    log = pd.read_csv(dataset_name)
    filtered_log = filter_log_efficiently(log, min_len=80)  # Example: filter traces with length >= 100
    
    # Compute statistics
    n_traces = filtered_log['case:concept:name'].nunique()
    avg_trace_len = filtered_log.groupby('case:concept:name').size().mean()
    
    # Note: Optimal cost computation would depend on your specific implementation
    # This is a placeholder

    return {
        'Dataset': dataset_name,
        'Traces (#)': n_traces,
        'Avg Tr Len': round(avg_trace_len, 1),
    }

In [34]:
compute_dataset_stats('Road_Traffic_Preprocessed.csv')

{'Dataset': 'Road_Traffic_Preprocessed.csv',
 'Traces (#)': 0,
 'Avg Tr Len': nan}

In [ ]:
compute_dataset_stats()

In [13]:

os.chdir(PATH)

In [29]:
sepsis_astar = pd.read_csv('Sepsis_Preprocessed2.csv_astar_results.csv')
sepsis_astar.describe()

,case_id,cost,time,nodes_popped
count,5.000000,5.000000,5.000000,5.000000
mean,476.200000,0.000130,23.344506,2164.600000
std,212.108934,0.000048,10.712041,1338.368148
min,237.000000,0.000080,14.433430,1097.000000
25%,378.000000,0.000090,15.698120,1358.000000
50%,464.000000,0.000120,16.488140,1455.000000
75%,490.000000,0.000170,35.010550,2581.000000
max,812.000000,0.000190,35.092290,4332.000000


In [30]:
sepsis_reach = pd.read_csv('Sepsis_Preprocessed2.csv_reach_results.csv')
sepsis_reach.describe()

,case_id,cost,time,nodes_popped
count,5.000000,5.000000,5.000000,5.000000
mean,476.200000,0.000130,0.857986,3140.200000
std,212.108934,0.000048,0.670662,1743.903724
min,237.000000,0.000080,0.356450,1984.000000
25%,378.000000,0.000090,0.386330,2008.000000
50%,464.000000,0.000120,0.475820,2112.000000
75%,490.000000,0.000170,1.174690,3576.000000
max,812.000000,0.000190,1.896640,6021.000000


In [31]:
sepsis_our = pd.read_csv('Sepsis_Preprocessed2_results.csv')
sepsis_our.describe()

,case_id,window_15_markings_1_cost,window_15_markings_1_time,window_15_markings_1_nodes_opened,window_30_markings_1_cost,window_30_markings_1_time,window_30_markings_1_nodes_opened,window_50_markings_1_cost,window_50_markings_1_time,window_50_markings_1_nodes_opened
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,476.200000,0.000130,0.401096,5628.800000,0.000130,0.495226,5628.800000,0.000130,0.607186,5628.800000
std,212.108934,0.000048,0.177328,2513.727253,0.000048,0.217542,2513.727253,0.000048,0.291780,2513.727253
min,237.000000,0.000080,0.177310,2689.000000,0.000080,0.221840,2689.000000,0.000080,0.266970,2689.000000
25%,378.000000,0.000090,0.327190,3712.000000,0.000090,0.388610,3712.000000,0.000090,0.363990,3712.000000
50%,464.000000,0.000120,0.355250,5438.000000,0.000120,0.450540,5438.000000,0.000120,0.639850,5438.000000
75%,490.000000,0.000170,0.505310,7849.000000,0.000170,0.633360,7849.000000,0.000170,0.802180,7849.000000
max,812.000000,0.000190,0.640420,8456.000000,0.000190,0.781780,8456.000000,0.000190,0.962940,8456.000000


In [32]:
sepsis_our_h = pd.read_csv('Sepsis_Preprocessed2_results_h.csv')
sepsis_our_h.describe()

,case_id,window_15_markings_1_cost,window_15_markings_1_time,window_15_markings_1_nodes_opened,window_30_markings_1_cost,window_30_markings_1_time,window_30_markings_1_nodes_opened,window_50_markings_1_cost,window_50_markings_1_time,window_50_markings_1_nodes_opened
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,476.200000,0.000130,0.284762,2350.000000,0.000130,0.338172,2749.000000,0.000130,0.404370,3011.000000
std,212.108934,0.000048,0.177216,909.433065,0.000048,0.139890,945.679121,0.000048,0.204196,933.653576
min,237.000000,0.000080,0.157400,1539.000000,0.000080,0.204060,2034.000000,0.000080,0.258200,2322.000000
25%,378.000000,0.000090,0.182000,1756.000000,0.000090,0.237890,2242.000000,0.000090,0.271920,2364.000000
50%,464.000000,0.000120,0.222640,2230.000000,0.000120,0.290610,2364.000000,0.000120,0.295820,2395.000000
75%,490.000000,0.000170,0.269210,2364.000000,0.000170,0.415030,2725.000000,0.000170,0.454530,3594.000000
max,812.000000,0.000190,0.592560,3861.000000,0.000190,0.543270,4380.000000,0.000190,0.741380,4380.000000


In [ ]:
def run_algorithm_with_timeout(sync_prod, algorithm: str, timeout: Optional[int] = None) -> Optional[Dict[str, Any]]:
    """
    Runs a specified search algorithm on the provided sync product with a timeout.
    Handles exceptions and returns None if the algorithm fails or times out.
    Includes the computation time in the results.
    """
    result = {}
    
    def target():
        try:
            start_time = time.time()  # Start the timer
            if algorithm == 'astar':
                result['alignment'], result['cost'], result['nodes_popped'] = sync_prod.astar_search()
            elif algorithm == 'astar_extended':
                result['alignment'], result['cost'], result['nodes_popped'] = sync_prod.astar_incremental()
            elif algorithm == 'reach':
                result['alignment'], result['cost'], result['nodes_popped'] = sync_prod.reach_search()
            end_time = time.time()  # End the timer
            result['time'] = end_time - start_time  # Calculate the elapsed time
        except Exception as e:
            result['error'] = e
    
    thread = Thread(target=target)
    thread.start()
    thread.join(timeout=timeout)
    
    if thread.is_alive():
        print(f"Timeout occurred for {algorithm}.")
        return None
    
    if 'error' in result:
        print(f"Error occurred for {algorithm}: {result['error']}")
        return None
    
    return {
        'cost': result.get('cost'),
        'nodes_popped': result.get('nodes_popped'),
        'time': result.get('time')
    }